# RDS to Redshift Lab

### Introduction

Now that we know about pulling data from RDS into S3, let's see if we can complete the loop that we've been seeing in our diagram.

<img src="./rds_ec2_s3.jpg" width="70%">

Remember that our data will begin in our RDS database, as this is the database that gets filled from users interacting with our. application.  Then we'll use our ETL server (here our laptop) to copy data from our RDS onto our ETL server as a csv file, and then place it into S3.  

Then, with our data properly exported as a csv file, we can move it into redshift.  Ok, let's get going.

### Setting up our RDS Database

If you do not have one already, create an RDS database that is publicly accessible.  And from there, we'll need to create tables for our foursquare application by running the `migrations/create_tables.sql` file against the RDS database.

After running the migrations we should see the following tables in our OLTP database.

> <img src="./oltp_tables.png" width="60%">

Ok, now generally our OLTP database will get data from our users, but for the purposes of this lab, we'll need to load in that data.  The data to load in is currently located in the directory `foursquare-fullstack/data`.

`COPY table_name
FROM 'absolute/path/to/data.csv' 
DELIMITER ',' 
CSV HEADER;`

For example, with the `states.csv` file in the Documents folder, we could login to our RDS shell, and load in our states data with something like the following.

<img src="./import-in-data.png" width="80%">

The `COPY 3` tells us that three rows were successfully copied into the states table.  Now copy over the `states.csv` file to the correct table, as well as the other CSV files. 

Once all of the data is properly loaded, then we can move onto the next step which is to export data to a CSV file onto our laptop that we can then load into S3.  We'll do that in the next section.

### Loading data to a csv file

Now it's time to load data from our OLTP in our RDS onto our ETL server so that we can then move that data into csv files to be loaded into our OLAP on redshift.  In other words, in the diagram below, our next step is the first arrow on the left.

<img src="./rds_ec2_s3.jpg" width="70%">

Now when we copy the data into our csv file, we should do so in a way so that we can ultimately load our data to our OLAP schema.  In other words, when exporting our data, we should perform ETL in SQL so that our data is lines up with our star schema.

> <img src="./updated_star.png" width="40%">

So we'll need to export our data so that it lines up to a `venues.csv`, `locations.csv`, and `categories.csv` file as illustrated above, with the header included in the csv.

> It's probably best to first practice an ordinary `SELECT` statement to make sure we are selecting the correct data, and then combine that with the correct `\copy` command.

> **Note**: Remember that when exporting data, the primary key in our csv file should align with the primary key in our OLTP tables.

The first few rows of the files on your computer should look like the following:

* `venues.csv`

```csv
id,name,id,price,rating,likes
53,Los Tacos Al Pastor,50,1,,
54,Grimaldis,51,2,2,3
```

* `categories.csv`

```csv
id,name,venue_id
68,Pizza,54
69,Italian,54
70,Italian,55
```

* `locations.csv`

```csv
id,longitude,latitude,address,zipcode,city,state
50,40.7024,-73.9875,141 Front Street,11210,New York,New York
51,40.7024,-73.9875,1 Front Street,11210,New York,New York
52,40.7024,-73.9875,133 Wythe Avenue,10001,New York,New York
53,40.7024,-73.9875,237 James Street,19019,Pennsylvania,Pennsylvania
```

### Loading Data to S3

Now that we have the csv files on our ETL server, it's time to load them to S3.  If you have not already, create an S3 bucket and make the bucket publicly accessible with the appropriate permissions.  

Then, the next step is to use the aws CLI to upload files from our local computer and into that S3 bucket.

> If not currently logged into the aws CLI, we may have to visit the security credentials, and create a new access key, and then use the information to login to the command line via the `aws configure` command.

> <img src="./security-credentials.png" width="30%">

Ok, once logged in, call the appropriate aws command.  

> For example, when uploading our `states.csv` file in the reading we used the following command.  

> `aws s3 cp states.csv s3://jigsaw-sample-data/states.csv --acl public-read`

We can confirm that we issued the commands correctly by visiting our s3 bucket.

<img src="./uploaded-to-s3.png" width="40%">

Ok, so now we have completed the second step.  What's left is to move our data over to redshift.  Let's get to it.

### Loading Data into Redshift

In [71]:
import psycopg2
host = "redshift-cluster-1.cdpgnoufdsdf.us-east-1.redshift.amazonaws.com"
conn = psycopg2.connect(host = host,
                        database="dev",
                        port = "5439",
                        user="awsuser",
                        password="Password1")

In [72]:
cursor = conn.cursor()

> <img src="./updated_star.png" width="40%">

In [37]:
cursor.execute("DROP TABLE categories;")

In [38]:
conn.commit()

In [24]:
cursor.execute("DROP TABLE venues;")

In [25]:
conn.commit()

In [11]:
create_locations_command = """CREATE TABLE "locations" (
    "id" integer NOT NULL DEFAULT nextval('locations_id_seq'),
    "latitude" FLOAT,
    "longitude" FLOAT,
    "address" VARCHAR(200),
    "zipcode" VARCHAR(20),
    "city" VARCHAR(100),
    "state" VARCHAR(100)
);"""

In [15]:
cursor.execute(create_locations_command)

In [16]:
conn.commit()

In [39]:
create_categories_command = """CREATE TABLE "categories" (
    "id" integer NOT NULL DEFAULT nextval('categories_id_seq'),
    "name" VARCHAR(200),
    "venue_id" INTEGER
);"""

In [40]:
cursor.execute(create_categories_command)

In [41]:
conn.commit()

In [9]:
create_venues_command = """CREATE TABLE "venues" (
    "id" integer NOT NULL DEFAULT nextval('venues_id_seq'),
    "name" VARCHAR(200),
    "location_id" INTEGER,
    "price" INTEGER,
    "rating" FLOAT,
    "likes" INTEGER
);"""

In [29]:
cursor.execute(create_venues_command)

In [30]:
conn.commit()

After creating the tables, let's load in the data from our s3 bucket beginning with `categories`.

> We'll place the query for viewing the `stl_load_errors` in case it's needed.

In [ ]:
qry = """select query, substring(filename,22,25) as filename,line_number as line, 
substring(colname,0,12) as column, type, position as pos, substring(raw_line,0,50) as line_text,
substring(raw_field_value,0,15) as field_text, 
substring(err_reason,0,45) as reason
from stl_load_errors 
order by query desc
limit 1;"""

Ok, get started.

In [84]:
qry = """COPY categories (id, name, venue_id) from 's3://jk-jigsaw-foursquare/categories.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
IGNOREHEADER 1
region 'us-east-1';
"""

cursor.execute(qry)

In [85]:
conn.commit()

> Then we can confirm that our categories loaded properly.

In [86]:
cursor.execute('SELECT * FROM categories LIMIT 2;')

In [87]:
cursor.fetchall()

# [(68, 'Pizza', 54), (69, 'Italian', 54)]

[(68, 'Pizza', 54), (69, 'Italian', 54)]

Next up is to load in the data for the `locations` table.

In [88]:
qry = """COPY locations (id, latitude, longitude, address, zipcode, city, state) from 's3://jk-jigsaw-foursquare/locations.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
IGNOREHEADER 1
CSV QUOTE '\"'
region 'us-east-1';
"""

cursor.execute(qry)

In [89]:
conn.commit()

Then let's check some of the entries in the `locations` table.

In [90]:
cursor.execute("SELECT * FROM locations LIMIT 2;")
cursor.fetchall()
# [(50, 40.7024, -73.9875, '141 Front Street', '11210', 'New York', 'New York'),
#  (51, 40.7024, -73.9875, '1 Front Street', '11210', 'New York', 'New York')]

[(50, 40.7024, -73.9875, '141 Front Street', '11210', 'New York', 'New York'),
 (51, 40.7024, -73.9875, '1 Front Street', '11210', 'New York', 'New York')]

And finally, it's time to load in data for our venues table.

In [91]:
qry = """COPY venues (id, name, location_id, price, rating, likes) from 's3://jk-jigsaw-foursquare/venues.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/redshiftRole'
delimiter ','
IGNOREHEADER 1
CSV QUOTE '\"'
region 'us-east-1';
"""

cursor.execute(qry)

In [92]:
conn.commit()

In [93]:
cursor.execute('SELECT * FROM venues LIMIT 2;')

In [94]:
cursor.fetchall()
# [(53, 'Los Tacos Al Pastor', 50, 1, None, None),
#  (54, 'Grimaldis', 51, 2, 2.0, 3)]

[(53, 'Los Tacos Al Pastor', 50, 1, None, None),
 (54, 'Grimaldis', 51, 2, 2.0, 3)]

And now that we've completed the cycle, it's worth at least making one query against our redshift database.  Let's find the venue name and address of `Grimaldis`.

> <img src="./updated_star.png" width="40%">

In [99]:
cursor.execute("""SELECT venues.name, locations.address FROM venues JOIN locations 
ON venues.location_id = locations.id WHERE name = 'Grimaldis';""")

In [100]:
cursor.fetchall()

# [('Grimaldis', '1 Front Street')]

[('Grimaldis', '1 Front Street')]

There we go.  That feels good.

### Summary

In this lesson, we saw the full cycle of beginning with our data in our OLTP, copying the transformed data over to CSV files on our ETL server, and using the AWS CLI to move the data over to S3.  From there, we then connected to our redshift database, created the necessary tables for our star schema and copied over our data from S3.

<img src="./rds_ec2_s3.jpg" width="70%">